In [12]:
import sys
import pathlib
import numpy as np
import pandas as pd
sys.path.insert(0, "../0.data-download/scripts/")
from data_loader import load_data
from sklearn.model_selection import train_test_split

In [13]:
# load all of the data 
data_directory = "../0.data-download/data/"
dfs = load_data(data_directory, adult_or_pediatric = "all")
dependency_df = dfs[1]
sample_df = dfs[0]

/home/markw/gene_dependency_representations/1.data-exploration/../0.data-download/scripts/data_loader.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  dependency_df = pd.read_csv(dependency_data_file, index_col=0).reset_index().dropna(1)


In [14]:
groups = sample_df.groupby("age_categories")
df_list = []
for name, df in groups:
    
    # only looking for samples that contain Adult or Pediatric information
    if name == "Adult" or name == "Pediatric":
        df_list.append(df)
        
# merge dataframes through concatentation 
new_df = pd.concat(df_list, axis=0)
new_df

,DepMap_ID,cell_line_name,stripped_cell_line_name,CCLE_Name,alias,COSMICID,sex,source,RRID,WTSI_Master_Cell_ID,...,default_growth_pattern,model_manipulation,model_manipulation_details,patient_id,parent_depmap_id,Cellosaurus_NCIt_disease,Cellosaurus_NCIt_id,Cellosaurus_issues,age_categories,age_distribution
0,ACH-000076,NCO2,NCO2,NCO2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Female,HSRRB,CVCL_3043,NaN,...,NaN,NaN,NaN,PT-Ugji7b,NaN,"Chronic myelogenous leukemia, BCR-ABL1 positive",C3174,NaN,Adult,35.0
1,ACH-000077,MJ,MJ,MJ_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Male,ATCC,CVCL_1414,NaN,...,2D: suspension,NaN,NaN,PT-brIUSU,NaN,Primary cutaneous T-cell non-Hodgkin lymphoma,C3467,NaN,Adult,50.0
2,ACH-000093,Panc 05.04,PANC0504,PANC0504_PANCREAS,NaN,NaN,Female,ATCC,CVCL_1637,NaN,...,2D: adherent,NaN,NaN,PT-aJgS51,NaN,Pancreatic ductal adenocarcinoma,C9120,NaN,Adult,77.0
4,ACH-000216,JH-EsoAd1,JHESOAD1,JHESOAD1_OESOPHAGUS,NaN,NaN,Male,Academic lab,CVCL_8098,NaN,...,NaN,NaN,NaN,PT-Y4syw7,NaN,Barrett adenocarcinoma,C7027,NaN,Adult,66.0
5,ACH-000288,BT-549,BT549,BT549_BREAST,NaN,905951.0,Female,ATCC,CVCL_1092,1835.0,...,2D: adherent,NaN,NaN,PT-hgWgpg,NaN,Invasive breast carcinoma of no special type,C4194,NaN,Adult,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,ACH-002278,NB10,NB10,NB10_AUTONOMIC_GANGLIA,NaN,949171.0,Male,Sanger,CVCL_1441,1033.0,...,NaN,NaN,NaN,PT-1wIoRq,NaN,Neuroblastoma,C3270,NaN,Pediatric,2.0
1074,ACH-002280,NB13,NB13,NB13_AUTONOMIC_GANGLIA,NaN,949177.0,Male,Sanger,CVCL_1443,215.0,...,NaN,NaN,NaN,PT-pJmC9b,NaN,Neuroblastoma,C3270,NaN,Pediatric,1.0
1075,ACH-002282,NB17,NB17,NB17_AUTONOMIC_GANGLIA,NaN,949175.0,Female,Sanger,CVCL_1445,159.0,...,NaN,NaN,NaN,PT-KDwcNS,NaN,Neuroblastoma,C3270,NaN,Pediatric,1.0
1076,ACH-002283,NB5,NB5,NB5_AUTONOMIC_GANGLIA,NaN,949176.0,Female,Sanger,CVCL_8822,2071.0,...,2D: adherent,NaN,NaN,PT-x400Sq,NaN,Neuroblastoma,C3270,NaN,Pediatric,2.0


In [15]:
# creating a sample data frame that onl contains the desired information of sex and age category
ref_df = new_df[["DepMap_ID", "sex", "age_categories"]]

In [16]:
# sample data frame containing ALL the PEDIATRIC samples
bulk_pediatric_training_df = ref_df.loc[ref_df["age_categories"] == "Pediatric"].reset_index(drop=True)
print(bulk_pediatric_training_df.shape)
bulk_pediatric_training_df.head(3)

(121, 3)


,DepMap_ID,sex,age_categories
0,ACH-000172,Male,Pediatric
1,ACH-000602,Female,Pediatric
2,ACH-001099,Female,Pediatric


In [17]:
# searching for similar IDs FROM the PEDIATRIC-samples IN the gene_dependency df
presplit_pediatric_gene_dependency_IDs = bulk_pediatric_training_df["DepMap_ID"].tolist()
adult_IDs = set(presplit_pediatric_gene_dependency_IDs) & set(dependency_df["DepMap_ID"].tolist())

# creating a new gene dependency data frame containing only PEDIATRIC samples
presplit_pediatric_df = dependency_df.loc[dependency_df["DepMap_ID"].isin(presplit_pediatric_gene_dependency_IDs)].reset_index(drop=True)
print(presplit_pediatric_df.shape)
presplit_pediatric_df.head(3)

(121, 16709)


,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZUP1 (221302),ZW10 (9183),ZWILCH (55055),ZWINT (11130),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000039,0.005638,0.013047,0.025001,0.000525,0.006821,0.304223,0.016122,0.190903,0.006395,...,0.026922,0.069586,0.055861,0.598261,0.152682,0.035298,0.111591,0.011503,0.029215,0.429314
1,ACH-000045,0.032071,0.009133,0.016520,0.018493,0.015912,0.037355,0.107148,0.265936,0.049798,...,0.052046,0.155991,0.139282,0.836257,0.012569,0.053228,0.025654,0.040709,0.096456,0.408855
2,ACH-000052,0.078182,0.026625,0.007138,0.012611,0.036226,0.104041,0.007932,0.206539,0.005796,...,0.019637,0.225646,0.011741,0.762200,0.006665,0.005277,0.092042,0.100434,0.036555,0.486059


In [18]:
# sample data frame containing ALL the ADULT samples
bulk_adult_training_df = ref_df.loc[ref_df["age_categories"] == "Adult"].reset_index(drop=True)
print(bulk_adult_training_df.shape)
bulk_adult_training_df.head(3)

(763, 3)


,DepMap_ID,sex,age_categories
0,ACH-000076,Female,Adult
1,ACH-000077,Male,Adult
2,ACH-000093,Female,Adult


In [19]:
# searching for similar IDs FROM the ADULT-samples IN the gene_dependency df
presplit_adult_gene_dependency_IDs = bulk_adult_training_df["DepMap_ID"].tolist()
adult_IDs = set(presplit_adult_gene_dependency_IDs) & set(dependency_df["DepMap_ID"].tolist())

# creating a new gene dependency data containing only adult samples
presplit_adult_df = dependency_df.loc[dependency_df["DepMap_ID"].isin(presplit_adult_gene_dependency_IDs)].reset_index(drop=True)
print(presplit_adult_df.shape)
presplit_adult_df.head(3)

(763, 16709)


,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZUP1 (221302),ZW10 (9183),ZWILCH (55055),ZWINT (11130),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,0.094568,0.012519,0.027460,0.025962,0.073412,0.027340,0.020199,0.284733,0.022084,...,0.028997,0.555867,0.037449,0.080585,0.004241,0.082956,0.012000,0.003592,0.012679,0.324623
1,ACH-000004,0.012676,0.049011,0.075933,0.033215,0.013176,0.097497,0.005015,0.153166,0.007358,...,0.087655,0.007427,0.038768,0.230569,0.011203,0.060266,0.128375,0.005911,0.004645,0.042530
2,ACH-000005,0.053957,0.027968,0.010139,0.005448,0.018599,0.081636,0.005457,0.159904,0.050884,...,0.231759,0.071035,0.017479,0.274568,0.033416,0.034712,0.092832,0.012482,0.020843,0.050412


In [20]:
# train test split the adult gene dependency data
adult_train, adult_test = train_test_split(presplit_adult_df, test_size=0.15)

# train test split the pediatric gene dependency data
pediatric_train, pediatric_test = train_test_split(presplit_pediatric_df, test_size=0.15)

In [21]:
# merge pediatric and adult TRAINING frames
training_frames = [adult_test, adult_train]
training_df = pd.concat(training_frames).reset_index(drop=True)
print(training_df.shape)
training_df

(763, 16709)


,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZUP1 (221302),ZW10 (9183),ZWILCH (55055),ZWINT (11130),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000632,0.030450,0.043892,0.042842,0.083800,0.112787,0.020336,0.026740,0.220725,0.031826,...,0.088915,0.867530,0.650931,0.948203,0.087433,0.036298,0.056961,0.037912,0.018862,0.381143
1,ACH-000695,0.010058,0.015104,0.000773,0.004868,0.072825,0.091324,0.010135,0.243804,0.010446,...,0.005664,0.107168,0.041986,0.392883,0.012616,0.115573,0.123107,0.054825,0.070353,0.341790
2,ACH-002875,0.035519,0.099789,0.071748,0.048479,0.245413,0.154663,0.045155,0.324723,0.089083,...,0.048326,0.090844,0.329219,0.168367,0.017571,0.101216,0.087739,0.050947,0.182081,0.349872
3,ACH-001961,0.035764,0.028726,0.010083,0.008929,0.025282,0.077189,0.041049,0.111347,0.047064,...,0.029247,0.155142,0.437577,0.929468,0.033324,0.072580,0.056745,0.088812,0.014569,0.407301
4,ACH-000037,0.033884,0.007065,0.045215,0.012762,0.089646,0.031148,0.015177,0.657984,0.273457,...,0.011370,0.020136,0.006008,0.303327,0.072859,0.034732,0.058966,0.032731,0.110653,0.338194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,ACH-000487,0.260314,0.050457,0.082665,0.023382,0.051823,0.045040,0.010138,0.023786,0.146457,...,0.098275,0.299619,0.291877,0.308313,0.004344,0.042573,0.092451,0.017067,0.035072,0.159250
759,ACH-001551,0.048222,0.023507,0.064231,0.053847,0.205486,0.159143,0.067232,0.203397,0.043874,...,0.092122,0.065242,0.655720,0.627017,0.018121,0.029095,0.026031,0.043602,0.012996,0.326643
760,ACH-002046,0.127824,0.047530,0.041395,0.058535,0.135342,0.118283,0.028731,0.678783,0.051187,...,0.056723,0.443867,0.309965,0.944815,0.013072,0.067325,0.124884,0.187320,0.092736,0.563379
761,ACH-000805,0.011637,0.035847,0.030944,0.004527,0.119017,0.051007,0.045971,0.671822,0.225975,...,0.013418,0.174094,0.172765,0.844238,0.035642,0.126908,0.152868,0.135412,0.068852,0.665546


In [22]:
# save the TRAINING dataframe 
training_df_output = pathlib.Path("../0.data-download/data/VAE_train_df.csv")
training_df.to_csv(training_df_output)

In [23]:
# merge pediatric and adult TESTING frames
testing_frames = [pediatric_test, pediatric_train]
testing_df = pd.concat(testing_frames).reset_index(drop=True)
print(testing_df.shape)
testing_df

(121, 16709)


,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZUP1 (221302),ZW10 (9183),ZWILCH (55055),ZWINT (11130),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000341,0.050568,0.021249,0.019499,0.010081,0.084998,0.041638,0.051640,0.243288,0.022883,...,0.055409,0.276307,0.044960,0.566827,0.019762,0.099654,0.107253,0.051102,0.089868,0.349504
1,ACH-000233,0.038141,0.032545,0.052578,0.061856,0.030145,0.235902,0.139896,0.045093,0.064917,...,0.017716,0.757893,0.700784,0.950147,0.003129,0.033326,0.009356,0.027048,0.186785,0.298472
2,ACH-001735,0.046967,0.053750,0.021234,0.067052,0.059413,0.059950,0.022700,0.129932,0.013943,...,0.043679,0.227117,0.194011,0.913268,0.005348,0.027743,0.063005,0.067075,0.061076,0.250849
3,ACH-001053,0.039658,0.047216,0.036096,0.027867,0.050485,0.101331,0.009629,0.084776,0.007061,...,0.057507,0.747327,0.113929,0.795472,0.007090,0.055452,0.234586,0.184909,0.005498,0.450749
4,ACH-000995,0.284318,0.018971,0.031018,0.071332,0.144472,0.052292,0.003906,0.147992,0.088685,...,0.042101,0.047121,0.069629,0.459193,0.015786,0.009378,0.085567,0.006606,0.097257,0.564110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,ACH-000099,0.115981,0.036153,0.037712,0.040946,0.012724,0.178939,0.070937,0.017856,0.053938,...,0.145124,0.430725,0.326225,0.874960,0.006493,0.031134,0.177292,0.041216,0.089418,0.186312
117,ACH-000172,0.274377,0.063202,0.011664,0.007588,0.018075,0.170209,0.069151,0.853436,0.048085,...,0.023179,0.389114,0.231910,0.898018,0.051411,0.049795,0.077254,0.127839,0.536338,0.237384
118,ACH-001196,0.024321,0.419234,0.073319,0.007744,0.112050,0.254897,0.042257,0.409224,0.050138,...,0.122066,0.024205,0.071529,0.498776,0.376140,0.065758,0.085007,0.055441,0.255157,0.263931
119,ACH-001028,0.041097,0.094812,0.031027,0.025604,0.055969,0.115373,0.067750,0.134096,0.065525,...,0.050953,0.103624,0.256184,0.866614,0.039735,0.073950,0.057607,0.018593,0.120312,0.577451


In [24]:
# save the TESTING dataframe 
testing_df_output = pathlib.Path("../0.data-download/data/VAE_test_df.csv")
testing_df.to_csv(testing_df_output)